In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Oct 07 10:35:29 2016

@author: Job Verkaik, Mattijn van Hoek, HKV Lijn in water
"""
from os.path import isdir
#from fewslogger import Logger
import datetime
import sys, getopt, shutil
import netCDF4
import numpy as np
from xml.etree import ElementTree
from xml.dom import minidom
from xml.etree.ElementTree import Element, SubElement, Comment
import matplotlib.pyplot as plt
import pytz
%matplotlib inline

In [2]:
#Functions 
#=======================================================================================
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

def writeState(state_config_read, state_config_write, stateTime):
    """
    stateFile  ::  absolute path to write xml file
    stateTime  ::  in 'yyyy-mm-dd' format
    """
    eTree = ElementTree.parse(state_config_read).getroot()
    # start xml
    xml = Element('State')
    xml.set('xmlns','http://www.wldelft.nl/fews')
    xml.set('xmlns:xsi','http://www.w3.org/2001/XMLSchema-instance')
    xml.set('xsi:schemaLocation','http://www.wldelft.nl/fews/PI http://fews.wldelft.nl/schemas/version1.0/pi-schemas/pi_state.xsd')
    xml.set('version','1.2')

    stateID = SubElement(xml,'stateId').text = eTree[0].text
    timeZone = SubElement(xml,'timeZone').text = eTree[1].text
    dateTime = SubElement(xml,'dateTime')
    dateTime.set('date',stateTime)
    dateTime.set('time',eTree[2].attrib.get('time'))
    stateLoc = SubElement(xml,'stateLoc')
    stateLoc.set('type','file')
    readLocation = SubElement(stateLoc, 'readLocation').text = eTree[3][0].text
    writeLocation = SubElement(stateLoc, 'writeLocation').text = eTree[3][1].text

    # save xml to file
    with open(state_config_write, 'w') as the_file:
        the_file.write(prettify(xml))   

def createNETCDF(filename, in_nc_arr="", STATE_COPY_CREATE=0, TIME_LIST=[], HM_ARRAY=[], fillValue = -999.0, state_config_read="exportStateConfig.xml", state_config_write="importStateConfig.xml"):
    """
    filename           ::  path to new NETCDF file
    in_nc_arr          ::  base NETCDF file
    STATE_COPY_CREATE  ::  set 0 for state
                           set 1 for copy
                           set 2 for create
    TIME_LIST          ::  when creating, provide datetime array with dates
    HM_ARRAY           ::  provide array with h.m data to create NETCDF
    fillValue          ::  only used during creation [default: -999.0] 
    stateConfigFile    ::  only used for writing state files [default: "%s/importStateConfig.xml" % statedir]    
    """
    # 0 is TimeSeriesSet STATE FILE
    # 1 is TimeSeriesSet COPY
    # 2 is TimeSeriesSet CREATE
    # ------------------------------------------            
    # create new warmState
    # create new.nc url and open file to write
    print("entered")
    new_nc = filename
    nc_new = netCDF4.Dataset(new_nc, 'w', format="NETCDF3_CLASSIC")

    # create dimensions for new nc file based on in.nc
    if STATE_COPY_CREATE == 0 :
        nt = 1
    elif STATE_COPY_CREATE == 1:
        nt = in_nc_arr.dimensions['time'].size
    elif STATE_COPY_CREATE == 2:
        nt = len(TIME_LIST)        
    ny = in_nc_arr.dimensions['y'].size
    nx = in_nc_arr.dimensions['x'].size
    nc_new.createDimension('time', nt)
    nc_new.createDimension('y', ny)
    nc_new.createDimension('x', nx)

    # copy over time variable from in.nc [only first slice]
    time_innc = in_nc_arr.variables['time']
    time = nc_new.createVariable('time', 'f8', ('time',))
    if STATE_COPY_CREATE == 0:
        print ("time is: "+str(time_innc[-1]))
        new = time_innc[-1]

        new_min = new * 60
        new_date = datetime.datetime.fromtimestamp(new_min)#.strftime('%Y-%m-%d')    
        new_date = datetime.datetime(*new_date.timetuple()[:3]) #+ datetime.timedelta(hours=1)

        up_dateISO = new_date - datetime.timedelta(days=30)
        print(3,"This is updateDepth.py: state_config_read: "+str(state_config_read))
        print(3,"This is updateDepth.py: state_config_write: "+str(state_config_write))
        print(3,"This is updateDepth.py: state_config_write: "+str(up_dateISO.strftime('%Y-%m-%d')))
        writeState(state_config_read, state_config_write, stateTime=up_dateISO.strftime('%Y-%m-%d'))
        up_date = datetime.datetime.timestamp(up_dateISO)
        print(3,"This is updateDepth.py: warmStateTime in ISO: "+str(up_date))
        up_min = up_date / 60                
        time[:] = up_min #time_innc[-1]
        
    elif STATE_COPY_CREATE == 1:
        time[:] = time_innc[:]
    elif STATE_COPY_CREATE == 2:
        time[:] = TIME_LIST
#     ws_epoch_min = time_innc[-1] * 60
#     ws_datetime = datetime.datetime.fromtimestamp(ws_epoch_min).strftime('%Y-%m-%d')
#     print(3,"This is updateDepth.py: warmStateTime in datetime: "+str(ws_datetime))
    time.standard_name = time_innc.standard_name
    time.long_name = time_innc.long_name
    time.units = time_innc.units
    time.axis = time_innc.axis

    # copy over y variable from in.nc
    y_innc = in_nc_arr.variables['y']
    y = nc_new.createVariable('y', 'f8', ('y',), fill_value = y_innc._FillValue ) 
    y[:] = y_innc[:]
    y.standard_name = y_innc.standard_name
    y.long_name = y_innc.long_name
    y.units = y_innc.units
    y.axis = y_innc.axis

    # copy over x variable from in.nc
    x_innc = in_nc_arr.variables['x']
    x = nc_new.createVariable('x', 'f8', ('x'), fill_value = x_innc._FillValue)
    x[:] = x_innc[:]
    x.standard_name = x_innc.standard_name
    x.long_name = x_innc.long_name
    x.units = x_innc.units
    x.axis = x_innc.axis

    # copy over z variable from in.nc
    z_innc = in_nc_arr.variables['z']
    z = nc_new.createVariable('z', 'f8', ('y', 'x'), fill_value = z_innc._FillValue)
    z[:] = z_innc[:]
    z.long_name = z_innc.long_name
    z.units = z_innc.units
    z.axis = z_innc.axis
    try:
        z.postive = z_innc.positive
    except:
        l = 1

    # copy over lat variable from in.nc
    lat_innc = in_nc_arr.variables['lat']
    lat = nc_new.createVariable('lat', 'f8', ('y', 'x'), fill_value = lat_innc._FillValue)
    lat[:] = lat_innc[:]
    lat.standard_name = lat_innc.standard_name
    lat.long_name = lat_innc.long_name
    lat.units = lat_innc.units

    # copy over lat variable from in.nc
    lon_innc = in_nc_arr.variables['lon']
    lon = nc_new.createVariable('lon', 'f8', ('y', 'x'), fill_value = lon_innc._FillValue)
    lon[:] = lon_innc[:]
    lon.standard_name = lon_innc.standard_name
    lon.long_name = lon_innc.long_name
    lon.units = lon_innc.units

    # copy over crs variable from in.nc
    crs_innc = in_nc_arr.variables['crs']
    crs = nc_new.createVariable('crs', 'i4', ())
    crs.long_name = crs_innc.long_name
    crs.crs_wkt = crs_innc.crs_wkt
    crs.proj4_params = crs_innc.proj4_params
    crs.epsg_code = crs_innc.epsg_code

    # copy over HM variable from in.nc [only first slice]
    HM_innc = in_nc_arr.variables['HM']
    if STATE_COPY_CREATE == 0:
        noDataVal = HM_innc._FillValue
    elif STATE_COPY_CREATE == 1:
        noDataVal = HM_innc._FillValue
    elif STATE_COPY_CREATE == 2:
        noDataVal = fillValue
    HM = nc_new.createVariable('HM', 'f8', ('time', 'y', 'x'), fill_value = noDataVal)
    HM[:] = HM_ARRAY[:]
    #HM[:] = hm_out[:]
    HM.long_name = HM_innc.long_name
    HM.units = HM_innc.units
    HM.coordinates = HM_innc.coordinates
    HM.grid_mapping = HM_innc.grid_mapping

    #close newly created warmState nc file
    nc_new.close()        
#=======================================================================================

In [3]:
state_config_read = r'D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\state/exportStateConfig.xml' 
state_config_write = r'D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\state/importStateConfig.xml'
stateTime = '2016-09-01'
writeState(state_config_read, state_config_write, stateTime)

In [4]:
#local = pytz.timezone ('Europe/Amsterdam')


In [6]:
# MAIN PROGRAM  
# def main(argv):
#     # input argument checking
#     try:
#         opts, args = getopt.getopt(argv,"hi:o:s:r:",["ipath=","spath=","opath=","rpath="])
#     except getopt.GetoptError:
#         print ('usage: updateDepth.py -i <inputdir> -s <statedir> -o <outputdir> -r <runfiledir>')
#         sys.exit(2)
#     for opt, arg in opts:
#         if opt == '-h':
#             print ('updateDepth.py -i <inputdir> -s <statedir> -o <outputdir> -r <runfiledir>')
#             sys.exit()
#         elif opt in ("-i", "--inputdir"):
#             inputdir = arg
#         elif opt in ("-s", "--statedir"):
#             statedir = arg
#         elif opt in ("-o", "--outputdir"):
#             outputdir = arg
#         elif opt in ("-r", "--runfiledir"):
#             runfiledir = arg            
inputdir = r'D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\input'
statedir = r'D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\state'
outputdir = r'D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\output'
runfiledir = r'D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\runfile'
diagnosticsfile = r'D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate//diagnostics.xml'

#     diagnosticsfile = "diagnostics.xml"
#     log = Logger(diagnosticsfile)
print(3,"This is updateDepth.py: doe iets slims; M. van Hoek &amp; J. Verkaik, HKV Lijn in water")
print(4,"Inputdir: %s" % inputdir)
print(4,"Statedir: %s" % statedir)
print(4,"Outputdir: %s" % outputdir)
print(4,"Runfiledir: %s" % runfiledir)    
print(4,"Diagnosticsfile: %s" % diagnosticsfile)


try:
    # DOE IETS SLIMS VANAF HIER

    #shutil.copyfile("%s/in.nc" % inputdir, "%s/out.nc" % outputdir)

    # load input netcdf and state netcdf
    # in_nc = r'D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate4pythontesting\input//in.nc'
    # state_nc = r'D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate4pythontesting\state//state.nc'
    # out_nc = r'D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate4pythontesting\output//out.nc'

    in_nc = "%s/in.nc" % inputdir
    state_nc = "%s/state.nc" % statedir
    state_config_read = "%s/exportStateConfig.xml" % statedir
    state_config_write = "%s/importStateConfig.xml" % statedir
    out_nc = "%s/out.nc" % outputdir
    print(3,"This is updateDepth.py: "+str(state_config_read)+" :: "+str(state_config_write))            
    print(in_nc, state_nc, out_nc)
    
    in_nc_arr = netCDF4.Dataset(in_nc, )
    state_nc_arr = netCDF4.Dataset(state_nc, )
    print(in_nc_arr)

    print(in_nc_arr.variables.keys()) # get all variable names
    hm = in_nc_arr.variables['HM']  # bodemhoogte peiling
    hm_state = state_nc_arr.variables['HM']  # coldState
    hm_state_time = state_nc_arr.variables['time']  # coldState
    cs_epoch_min = hm_state_time[0] * 60
    cs_datetime = datetime.datetime.fromtimestamp(cs_epoch_min).strftime('%Y-%m-%d')        
    print(3,"This is updateDepth.py: coldStateTime in ms since epoch: "+str(cs_datetime))

    # create copy in.nc
    hm_copy = np.copy(hm)
    hm_copy = np.ma.masked_equal(hm_copy, -999)

    # create deep copy in.nc
    hm_deep_copy = np.full(hm.shape, -999)
    hm_deep_copy = np.ma.masked_equal(hm_deep_copy, -999)       

    # use coldState to update first slice in.nc
    a = hm[0,::] 
    b = hm_state[0,::]
    # update first slice based on coldState
    a[~b.mask] = b.compressed()
    hm_copy[0,::] = a
    hm_deep_copy[0,::] = np.ma.copy(a)

    # create init warmState
    hm_out = hm_copy[0,::]

    for ix in range(hm.shape[0] - 1):    
        # print (ix)
        # one by one fill/update and ammend arrays
        # get first slice of copy, get second slice of original
        a = hm_copy[ix,::] 
        b = hm[ix+1,::]  

        # update first slice based on second slice
        a[~b.mask] = b.compressed()
        # update timeSeriesSet
        hm_copy[ix+1,::] = a
        hm_deep_copy[ix+1,::] = np.ma.copy(a)
        # update warmState
        hm_out[~a.mask] = a.compressed()          

    # ------------------------------------------            
    # create new warmState
    # create NETCDF file for new timeSeriesSet        
    createNETCDF(state_nc, in_nc_arr = in_nc_arr, STATE_COPY_CREATE = 0, HM_ARRAY = hm_out, state_config_read=state_config_read, state_config_write=state_config_write)



    # ------------------------------------------
    # create out.nc timeSeriesSet
    # create new.nc url and open file to write
    createNETCDF(out_nc, in_nc_arr = in_nc_arr, STATE_COPY_CREATE = 1, HM_ARRAY = hm_deep_copy )


except:
    try:
        print(3,"This is updateDepth.py: arrived exception, no new data after warmState")
        state_nc = "%s/state.nc" % statedir
        state_config_read = "%s/exportStateConfig.xml" % statedir
        state_config_write = "%s/importStateConfig.xml" % statedir        
        runfile_xml = "%s/runfile.xml" % runfiledir
        print(3,"This is updateDepth.py: "+str(state_config_read)+" :: "+str(state_config_write))        

        # set timezone naar GMT-1
        local = pytz.timezone ('Etc/GMT-1')
        
        state_nc_arr = netCDF4.Dataset(state_nc, )        
        hm_state = state_nc_arr.variables['HM']  # coldState
        hm_state_time = state_nc_arr.variables['time']  # coldState
        cs_epoch_min = hm_state_time[0] * 60
        cs_date = datetime.datetime.fromtimestamp(cs_epoch_min)#.strftime('%Y-%m-%d')    
        cs_date = datetime.datetime(*cs_date.timetuple()[:3]) #+ datetime.timedelta(hours=1)
        cs_date = local.localize(cs_date, is_dst=None)
        print(3,"This is updateDepth.py: date coldState runfile: "+str(cs_date.isoformat()))

        # open runfile.xml and get time0 date in datetime format
        e = ElementTree.parse(runfile_xml).getroot()
        time0 = e[3].attrib.get('date') # e[3] is time0 in xml file
        # get timezone element as integer
        timezone=int(float(e[0].text))

        # get t0 element as datetime object
        time0_date = e[3].attrib.get('date') # e[3] is time0 in xml file
        time0_date_list = time0_date.replace('-', ' ').split(' ')
        time0_time = e[3].attrib.get('time')
        time0_time_list = time0_time.replace(':', ' ').split(' ')
        time0_dt = datetime.datetime(int(time0_date_list[0]), # year
                                     int(time0_date_list[1]), # month
                                     int(time0_date_list[2]), # day                                
                                     int(time0_time_list[0]), # hour
                                     int(time0_time_list[1]), # minute
                                     int(time0_time_list[2]), # second
                                     ) 
        time0_dt = time0_dt + datetime.timedelta(hours=timezone)        
        time0_dt = datetime.datetime(*time0_dt.timetuple()[:3]) #+ datetime.timedelta(hours=1)
        time0_dt = local.localize(time0_dt, is_dst=None)

        print(3,"This is updateDepth.py: date time0_date runfile: "+str(time0_dt.isoformat())) 

        # get new datelist in MINUTES since epoch
        numdays = time0_dt - cs_date
        datesISO = [time0_dt - datetime.timedelta(days=x) for x in range(0, numdays.days + 1)]
        datesISO_isoString = [date.isoformat() for date in datesISO]
        print(3,"This is updateDepth.py: date time0_date runfile: "+str(datesISO_isoString)) 

        datesEpoch = []
        for date in datesISO:
            datesEpoch.append(date.timestamp() / 60.)    
        datesEpoch.reverse()
        # create numpy array of epoch
        #datesEpoch = np.array(datesEpoch)

        # repeat array N times where N is number of days between state and time0
        N = len(datesEpoch)
        A = np.array(hm_state[0,::])
        B = np.asarray([A]*N)

        # mask nodata values
        hm_out = np.ma.masked_equal(B, -999)

        # create NETCDF file for new timeSeriesSet        
        createNETCDF(out_nc, in_nc_arr = state_nc_arr, STATE_COPY_CREATE = 2, TIME_LIST = datesEpoch, HM_ARRAY = hm_out )

        # open out_NC as input for new WarmStateFile
        out_nc_arr = netCDF4.Dataset(out_nc, )    
        newHM_out = hm_out[-1]

        # create NETCDF file for new WarmStateFile        
        print(3,"upon entering")         
        createNETCDF(state_nc, in_nc_arr = out_nc_arr, STATE_COPY_CREATE = 0, HM_ARRAY = newHM_out, state_config_read=state_config_read, state_config_write=state_config_write)


    except Exception as e: 
        print (str(e))
        if not (isdir(inputdir)):
            print(1,"%s is not a valid path" % inputdir)
        elif not (isdir(statedir)):
            print(1,"%s is not a valid path" % statedir)
        elif not (isdir(outputdir)):
            print(1,"%s is not a valid path" % outputdir)
        elif not (isdir(runfiledir)):
            print(1,"%r is not a valid path" % runfiledir)
        else:
            print(1,"something else is funky")            

# TOT HIER
print(3,"Dat was het, FEWS take over please")
# log.close()
    
# if __name__ == "__main__":
#     main(sys.argv[1:])

3 This is updateDepth.py: doe iets slims; M. van Hoek &amp; J. Verkaik, HKV Lijn in water
4 Inputdir: D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\input
4 Statedir: D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\state
4 Outputdir: D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\output
4 Runfiledir: D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\runfile
4 Diagnosticsfile: D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate//diagnostics.xml
3 This is updateDepth.py: D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\state/exportStateConfig.xml :: D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\state/importStateConfig.xml
D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\input/in.nc D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\state/state.nc D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\output/out.nc
3 This is updateDepth.py: arrived exception, no new data after warmState
3 This is updateDepth.py: D:\OmsWaddenzee\trunk\fews\Modules\depthUpdate\state/exportStateConfig.xml :: D:\OmsWaddenzee\trunk\f

In [16]:
str(cs_date.isoformat())

'2017-02-28T00:00:00+01:00'

In [15]:
str(time0_dt)

'2017-03-30 00:00:00+01:00'

In [13]:
[date.isoformat() for date in datesISO]#datesISO#.isoformat()

['2017-03-30T00:00:00+01:00',
 '2017-03-29T00:00:00+01:00',
 '2017-03-28T00:00:00+01:00',
 '2017-03-27T00:00:00+01:00',
 '2017-03-26T00:00:00+01:00',
 '2017-03-25T00:00:00+01:00',
 '2017-03-24T00:00:00+01:00',
 '2017-03-23T00:00:00+01:00',
 '2017-03-22T00:00:00+01:00',
 '2017-03-21T00:00:00+01:00',
 '2017-03-20T00:00:00+01:00',
 '2017-03-19T00:00:00+01:00',
 '2017-03-18T00:00:00+01:00',
 '2017-03-17T00:00:00+01:00',
 '2017-03-16T00:00:00+01:00',
 '2017-03-15T00:00:00+01:00',
 '2017-03-14T00:00:00+01:00',
 '2017-03-13T00:00:00+01:00',
 '2017-03-12T00:00:00+01:00',
 '2017-03-11T00:00:00+01:00',
 '2017-03-10T00:00:00+01:00',
 '2017-03-09T00:00:00+01:00',
 '2017-03-08T00:00:00+01:00',
 '2017-03-07T00:00:00+01:00',
 '2017-03-06T00:00:00+01:00',
 '2017-03-05T00:00:00+01:00',
 '2017-03-04T00:00:00+01:00',
 '2017-03-03T00:00:00+01:00',
 '2017-03-02T00:00:00+01:00',
 '2017-03-01T00:00:00+01:00',
 '2017-02-28T00:00:00+01:00']

In [17]:
local = pytz.timezone ('Europe/Amsterdam')

In [ ]:
datetime.datetime(2017, 3, 30, 0, 0)

In [ ]:
from datetime import datetime
from dateutil import tz

In [ ]:
import datetime
utc_datetime = datetime.datetime.utcnow()
utc_datetime.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
datetime.datetime(2017, 3, 30, 0, 0)

In [ ]:
import time
time.daylight

In [ ]:
wintertime = 'W. Europe'# Standard Time'
summertime = 'W. Europe Daylight Time'

In [18]:
import pytz, datetime
#local = pytz.timezone ('Etc/GMT+0')
local = pytz.timezone ('Europe/Amsterdam')

In [24]:
naive = datetime.datetime.strptime ("2017-3-31 12:0:0", "%Y-%m-%d %H:%M:%S")
local_dt = local.localize(naive, is_dst=None)
utc_dt = local_dt.astimezone (pytz.utc)

In [26]:
local_dt#.isoformat()

datetime.datetime(2017, 3, 31, 12, 0, tzinfo=<DstTzInfo 'Europe/Amsterdam' CEST+2:00:00 DST>)

In [27]:
utc_dt

datetime.datetime(2017, 3, 31, 10, 0, tzinfo=<UTC>)

In [28]:
local_dt.isoformat()

'2017-03-31T12:00:00+02:00'